### Import libraries

In [1]:
import sys
import numpy as np
import os
import pandas as pd
import uproot
import time
import matplotlib.pyplot as plt
import awkward as ak
import matplotlib as mpl

Welcome to JupyROOT 6.24/02


### Import Simulation Data (Muons) (LNGS)

In [2]:
folder = "/home/jat89/project/muonVeto/muonFiles/"

colnames = ["ChainNumber", "ParticleInputEnergy", "ParticleDirectionX", "ParticleDirectionY", "ParticleDirectionZ", "Energy_Phase0BottomMVThickSolid"]
array_muons = uproot.concatenate(folder+"Muons_25*.root:qtree", colnames, library = 'np', allow_missing = True)#question mark wildcard

df_muons = pd.DataFrame(array_muons)

### Import Simulation Data (Gamma) (LNGS)

In [3]:
folder = "/home/jat89/project/muonVeto/gammaFiles/phase0/"

colnames = ["ChainNumber", "ParticleInputEnergy", "ParticleDirectionX", "ParticleDirectionY"
            , "ParticleDirectionZ", "Energy_Phase0BottomMVThickSolid"]
array = uproot.concatenate(folder+"Gammas_5*.root:qtree", colnames, library = 'np', allow_missing = True)#question mark wildcard

df = pd.DataFrame(array)



### Put Gamma and Muon Data into Separate Arrays

In [4]:
gamma_spectrum = []
muon_spectrum = []
for element in df.Energy_Phase0BottomMVThickSolid:
    if(len(element)==0):
        continue
    else:
        gamma_spectrum.append(element[0])
        
for element in df_muons.Energy_Phase0BottomMVThickSolid:
    if(len(element)==0):
        continue
    else:
        muon_spectrum.append(element[0])

### Import Simulation Data (Gamma) (Lab)

In [ ]:
folder_g = "gamma_sim/gamma_sim-build/" # location of the gamma sim build folder
folder_m = "muons_edited/muonsInPanel_build/" # location of the muon sim build folder
 
# Open ROOT files
f1460 = TFile.Open(folder_g + "K40-1460KeV-10x.root","READ")
f2614 = TFile.Open(folder_g + "Tl208-2614KeV-25x.root","READ")

# Bismuth peaks
f1120 = TFile.Open(folder_g + "Bi214-1120KeV-25x.root", "READ")
f1238 = TFile.Open(folder_g + "Bi214-1238KeV-25x.root", "READ")
f1764 = TFile.Open(folder_g + "Bi214-1764KeV-25x.root", "READ")
f2204 = TFile.Open(folder_g + "Bi214-2204KeV-25x.root", "READ")

# Lower peaks
f968 = TFile.Open(folder_g + "Ac228-968KeV-25x.root", "READ")
f934 = TFile.Open(folder_g + "Bi214-934KeV-25x.root", "READ")
f911 = TFile.Open(folder_g + "Ac228-911KeV-25x.root", "READ")
f609 = TFile.Open(folder_g + "Bi214-609KeV-25x.root", "READ")
f583 = TFile.Open(folder_g + "Tl208-583KeV-25x.root", "READ")

# Muons
muons = TFile(folder_m + "muonsJ_normalized.root", "READ")

## 

# Read the TTree for main two gamma isotopes
tree1460 = f1460.Get("Hits")
tree2614 = f2614.Get("Hits")

# Read the TTree for the nearby Bismuth-214 isotopes
tree1120 = f1120.Get("Hits")
tree1238 = f1238.Get("Hits")
tree1764 = f1764.Get("Hits")
tree2204 = f2204.Get("Hits")

# Read the TTree for the remaining low-freq. isotopes
tree968 = f968.Get("Hits")
tree934 = f934.Get("Hits")
tree911 = f911.Get("Hits")
tree609 = f609.Get("Hits")
tree583 = f583.Get("Hits")

# For the TTree for Muons
a = muons.Get("ntuple")
muonstree = a.Get("parTup")

In [ ]:
# Append the energy values from the TTree into an array
energyDeposited1460 = []
entries= tree1460.GetEntries();
for i in range(0,entries):
    tree1460.GetEntry(i)
    energyDeposited1460.append(tree1460.EnergyDeposited)

energyDeposited2614 = []
entries= tree2614.GetEntries();
for i in range(0,entries):
    tree2614.GetEntry(i)
    energyDeposited2614.append(tree2614.EnergyDeposited)

energyDeposited1120 = []
entries= tree1120.GetEntries();
for i in range(0,entries):
    tree1120.GetEntry(i)
    energyDeposited1120.append(tree1120.EnergyDeposited)

energyDeposited1238 = []
entries= tree1238.GetEntries();
for i in range(0,entries):
    tree1238.GetEntry(i)
    energyDeposited1238.append(tree1238.EnergyDeposited)

energyDeposited1764 = []
entries= tree1764.GetEntries();
for i in range(0,entries):
    tree1764.GetEntry(i)
    energyDeposited1764.append(tree1764.EnergyDeposited)

energyDeposited2204 = []
entries= tree2204.GetEntries();
for i in range(0,entries):
    tree2204.GetEntry(i)
    energyDeposited2204.append(tree2204.EnergyDeposited)
    
# And the remaining peaks
energyDeposited968 = []
entries= tree968.GetEntries();
for i in range(0,entries):
    tree968.GetEntry(i)
    energyDeposited968.append(tree968.EnergyDeposited)

energyDeposited934 = []
entries= tree934.GetEntries();
for i in range(0,entries):
    tree934.GetEntry(i)
    energyDeposited934.append(tree934.EnergyDeposited)

energyDeposited911 = []
entries= tree911.GetEntries();
for i in range(0,entries):
    tree911.GetEntry(i)
    energyDeposited911.append(tree911.EnergyDeposited)

energyDeposited609 = []
entries= tree609.GetEntries();
for i in range(0,entries):
    tree609.GetEntry(i)
    energyDeposited609.append(tree609.EnergyDeposited)

energyDeposited583 = []
entries= tree583.GetEntries();
for i in range(0,entries):
    tree583.GetEntry(i)
    energyDeposited583.append(tree583.EnergyDeposited)
    
# Append muons
muonsEnergy = []
for event in muonstree:
    muonsEnergy.append(event.Edeposited)

### Import Lab Data

In [ ]:
%%capture

sourceFolder = "/home/iz43/GammaBackgroundElimination_Iffat/"

onlyfiles = [f for f in os.listdir(sourceFolder) if '.dat' in f]
print(onlyfiles)
data_list = []
for file in onlyfiles:
    data_list.append(dt5202_crunch(str(sourceFolder)+str(file), num_ev=100000000, num_ch=1))

data = data_list[0]
for i in range(1, len(data_list)):
    data = np.append(data, data_list[i])

In [ ]:
# Convert lab data to MeV

d = data['data']['low_gain']-ped
lab_mev = (d.astype('float64'))/adc_mev # lab data in MeV